### Here we take dataframe and split workplaces into smaller compartments

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from shapely.geometry import Point
from tqdm import tqdm
from geopy.distance import great_circle
import time as time
import random
from scipy.stats import binom

In [2]:
city_name = 'chelyabinsk'

input_path_people = '../data/initial_' + city_name + '/people_' + city_name + '_assigned_schools.txt'
input_path_workplaces = '../data/initial_' + city_name + '/workplaces_' + city_name + '.txt'

people = pd.read_csv(input_path_people, sep='\t', index_col=0)
workplaces = pd.read_csv(input_path_workplaces, sep='\t')


people['school_id'] = people['school_id'].str.replace('X','-1')
people['school_id'] = people['school_id'].astype(str).astype(float).astype(int)

people['work_id'] = people['work_id'].str.replace('X','-1')
people['work_id'] = people['work_id'].astype(str).astype(float).astype(int)

print(people.head())
print("Total number of people in population:", len(people['sp_id'].unique()))
print(workplaces.head())
print("Total number of workplaces in population", len(workplaces))
people['school_id'].unique()

   Unnamed: 0    sp_id  sp_hh_id  age sex  race  relate  school_id  work_id
0           0   599955         1    8   F     1       0         48       -1
1           1   790949         1   35   F     1       0         -1      173
2           2  1094293         1   69   F     1       0         -1       -1
3           3   556704         1    2   F     1       0         -1       -1
4           4   293648         1   39   M     1       0         -1      182
Total number of people in population: 1189525
   sp_id   latitude  longitude  size
0    1.0  55.186586  61.597408  25.0
1    2.0  55.159652  61.570179  31.0
2    3.0  55.177610  61.570179  41.0
3    4.0  55.195560  61.570179  41.0
4    5.0  55.213502  61.570179  28.0
Total number of workplaces in population 184


array([ 48,  -1,  44,  79,  99,  14,  52,  43, 105,  60, 109,   8,  54,
       101,  50,  64,  89,  59,  78,  84,  26,  61,  73,  96,  27,  95,
        51,  46,  69, 104,  40,  11,   2, 106,  72,  16,  80, 108,  17,
        20,  41,  82,  77,  67,   7, 107, 103,  30,  28,  42,  88,  25,
        83,  85,   5, 110,  65,   4,  49,  63,  32,  35,  53, 102,   9,
        29,  21,  37,  12,  62,  34,  75,  98,  90,  91,  13,  81,  10,
        24,  22,  92,  68,  87,  23,  39,   1, 100,  31,  18,  45, 111,
        86,  57,  58,  74,   6,  36,  15,  66,  97,   3,  56,  93,  71,
        47,  76,  33,  19,  94,  55,  38,  70])

In [3]:
def generate_distribution_data(sum, num_of_samples):
    data = []
    n_i = num_of_samples
    y_i = sum
    for i in range(num_of_samples):
        x_i = binom.rvs(y_i, 1/n_i, size=1)[0]
        data.append(x_i)
        y_i = y_i - x_i
        n_i = n_i - 1
    return data

work_id_old = people['work_id'].unique()
work_id_old = work_id_old
work_id_old = [int(float(elem)) for elem in work_id_old]
people_splitted = pd.DataFrame(columns=['sp_id', 'sp_hh_id', 'age',	'sex',	'race',	'relate', 'school_id', 'work_id'])
workplaces_splitted = pd.DataFrame(columns=['sp_id', 'latitude', 'longitude', 'size'])
work_id_max = 1

people_list = []

work_id_list = []
work_latitude_list = []
work_longitude_list = []
work_size_list = []

people_whole = people.copy()
people = people_whole[people['work_id'] > 0]
people_without_workplace = people_whole[people_whole['work_id'] < 0]
print(len(people_without_workplace), len(people))

551970 637555


In [4]:
for work_id in tqdm(work_id_old):
    if (work_id == -1):
        people_in_current_workplace = people[people['work_id'] == work_id]
        print(len(people_in_current_workplace))
        people_list.extend(list(people_in_current_workplace['sp_id']))
        for i in range(len(people_in_current_workplace)):
            work_id_list.append(-1)
    else:   
        people_in_current_workplace = people[people['work_id'] == work_id]
        people_list.extend(list(people_in_current_workplace['sp_id']))
        # print(list(people_in_current_dwelling['sp_id']))
        number_of_people_in_workplace = len(people_in_current_workplace)
        # print(people_list)
        
        
        if (number_of_people_in_workplace < 10):
            distribution_for_new_work_id = [number_of_people_in_workplace]
        else:
            distribution_for_new_work_id = generate_distribution_data(number_of_people_in_workplace, int(number_of_people_in_workplace/8.5))
        
        # distribution_for_new_hh_id = np.random.poisson(2.5, number_of_people_in_dwelling) 
        # print(number_of_people_in_workplace, sum(distribution_for_new_work_id))
        if (number_of_people_in_workplace != sum(distribution_for_new_work_id)):
            raise RuntimeError("Mismatch")
        
        for people_in_work_id in distribution_for_new_work_id:
            for i in range(people_in_work_id):
                work_id_list.append(work_id_max)
                work_size_list.append(people_in_work_id)
                work_latitude_list.append(float(workplaces[workplaces['sp_id'] == work_id].latitude))
                work_longitude_list.append(float(workplaces[workplaces['sp_id'] == work_id].longitude))
            work_id_max+=1

people_splitted = people
people_splitted['work_id'] = work_id_list
workplaces_splitted['sp_id'] = work_id_list
workplaces_splitted['size'] = work_size_list
workplaces_splitted['latitude'] = work_latitude_list
workplaces_splitted['longitude'] = work_longitude_list

  0%|                                                   | 0/185 [00:00<?, ?it/s]

0


/tmp/ipykernel_34624/2921711358.py:30: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  work_latitude_list.append(float(workplaces[workplaces['sp_id'] == work_id].latitude))
/tmp/ipykernel_34624/2921711358.py:31: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  work_longitude_list.append(float(workplaces[workplaces['sp_id'] == work_id].longitude))
100%|█████████████████████████████████████████| 185/185 [04:33<00:00,  1.48s/it]
/tmp/ipykernel_34624/2921711358.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  people_splitted['work_id'] = work_id_list


In [5]:
people_splitted = pd.concat([people_splitted, people_without_workplace])

people_splitted.to_csv('../data/initial_' + city_name + '/people_' + city_name + '_splitted_workplaces.txt', sep='\t')
workplaces_splitted.to_csv('../data/initial_' + city_name + '/workplaces_' + city_name + '_splitted.txt', sep='\t')

In [6]:
people_splitted['school_id'].unique()

array([ -1,  48,  44,  79,  99,  14,  52,  43, 105,  60, 109,   8,  54,
       101,  50,  64,  89,  59,  78,  84,  26,  61,  73,  96,  27,  95,
        51,  46,  69, 104,  40,  11,   2, 106,  72,  16,  80, 108,  17,
        20,  41,  82,  77,  67,   7, 107, 103,  30,  28,  42,  88,  25,
        83,  85,   5, 110,  65,   4,  49,  63,  32,  35,  53, 102,   9,
        29,  21,  37,  12,  62,  34,  75,  98,  90,  91,  13,  81,  10,
        24,  22,  92,  68,  87,  23,  39,   1, 100,  31,  18,  45, 111,
        86,  57,  58,  74,   6,  36,  15,  66,  97,   3,  56,  93,  71,
        47,  76,  33,  19,  94,  55,  38,  70])